# Futile Cycle Removal.

This notebook takes the draft *Lipomyces starkeyi* GSM and simulates growth on and extracts reactions that are generating redox/energy molecules such as ATP, NADPH, NADH to identify futile cycles. Reaction bounds are adjusted to block futile cycles. 

import needed libraries.

In [1]:
import cobra
import pandas
import numpy
from cobra.flux_analysis import pfba
from cobra.flux_analysis import gapfill

load GSM models.

In [2]:
# Lipomyces starkeyi constructed model (based on NRRL 11557 genome and 11558 phenotypic data).
lst = cobra.io.load_json_model('../models/Lst_v0.2_model_forPUB_copy.json')

# Rhodosporidium toruloides IFOxxxx.
rto = cobra.io.load_json_model('../models/Rt_IFO0880.json')

# Yarrowia lipolytica POxx.
ylip = cobra.io.load_matlab_model('../models/twoModels/iYLI647_corr.mat')

# Sacchromyces cerevisiae Yeast8.
sce = cobra.io.load_matlab_model('../models/yeast8_modifiedwBIGGnames.mat')


# E. coli iJO1366
eco = cobra.io.load_json_model('../models/iJO1366.json')

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"


This model seems to have confidenceScores instead of rxnConfidenceScores field. Will use confidenceScores for what rxnConfidenceScores represents.
This model seems to have metCharge instead of metCharges field. Will use metCharge for what metCharges represents.
No defined compartments in model model. Compartments will be deduced heuristically using regular expressions.
Using regular expression found the following compartments:c, e, g, m, n, r, v, x


In [3]:
r = lst.reactions.NADPHtru.copy()
r.id = 'NADHtru'
lst.add_reactions([r])
r.add_metabolites({'nadph_c': 1.0, 'nadph_r': -1.0,
                   'nadh_c': -1.0, 'nadh_r': 1.0 })
r = lst.reactions.NADPtru.copy()
r.id = 'NADtru'
lst.add_reactions([r])
r.add_metabolites({'nadp_r': 1.0, 'nadp_c': -1.0,
                   'nad_r': -1.0, 'nad_c': 1.0 })


In [4]:
# adjust iYLI647 parameters for consistent annotation. 
for m in ylip.metabolites:
    if ('[' in m.id):
        ylip.metabolites.get_by_id(m.id).id = m.id.replace('[','_').replace(']','')
    if ('_L_' in m.id):
        ylip.metabolites.get_by_id(m.id).id = m.id.replace('_L_','__L_')
    if ('_D_' in m.id):
        ylip.metabolites.get_by_id(m.id).id = m.id.replace('_D_','__D_')
for r in ylip.reactions:
    if ('(e)' in r.id):
        ylip.reactions.get_by_id(r.id).id = r.id.replace('(e)','_e')
    if(('_L_') in r.id):
        ylip.reactions.get_by_id(r.id).id = r.id.replace('_L_','__L_')

    if(('_D_') in r.id):
        ylip.reactions.get_by_id(r.id).id = r.id.replace('_D_','__D_')


view summary of the GSMs.

In [5]:
lst.reactions.EX_glc__D_e.lower_bound=-10
# lst.reactions.EX_xyl__D_e.upper_bound=0

lst.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.000508,0,0.00%
cu2_e,EX_cu2_e,0.0003206,0,0.00%
fe3_e,EX_fe3_e,0.003767,0,0.00%
glc__D_e,EX_glc__D_e,10,6,98.02%
k_e,EX_k_e,0.3576,0,0.00%
mg2_e,EX_mg2_e,0.03769,0,0.00%
mn2_e,EX_mn2_e,0.0003706,0,0.00%
na1_e,EX_na1_e,0.01594,0,0.00%
nh4_e,EX_nh4_e,5.158,0,0.00%
o2_e,EX_o2_e,17.76,0,0.00%


In [6]:
for r in lst.exchanges:
    if r.lower_bound!=0:
        print(r.id,r.bounds)

EX_h_e (-1000.0, 1000.0)
EX_h2o_e (-1000.0, 1000.0)
EX_nh4_e (-1000.0, 1000.0)
EX_o2_e (-1000.0, 1000.0)
EX_pi_e (-1000.0, 1000.0)
EX_so4_e (-1000.0, 1000.0)
EX_glc__D_e (-10, 1000.0)
EX_ca2_e (-1000.0, 1000.0)
EX_fe2_e (-1000.0, 1000.0)
EX_fe3_e (-1000.0, 1000.0)
EX_k_e (-1000.0, 1000.0)
EX_na1_e (-1000.0, 1000.0)
EX_mg2_e (-1000.0, 1000.0)
EX_mn2_e (-1000.0, 1000.0)
EX_cu2_e (-1000.0, 1000.0)
EX_zn2_e (-1000.0, 1000.0)


In [7]:
rto.reactions.EX_glc__D_e.lower_bound=-10

rto.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0006276,0,0.00%
cu2_e,EX_cu2_e,0.000396,0,0.00%
fe3_e,EX_fe3_e,0.004654,0,0.00%
glc__D_e,EX_glc__D_e,10,6,98.60%
k_e,EX_k_e,0.4418,0,0.00%
mg2_e,EX_mg2_e,0.04656,0,0.00%
mn2_e,EX_mn2_e,0.0004579,0,0.00%
na1_e,EX_na1_e,0.01969,0,0.00%
nh4_e,EX_nh4_e,4.979,0,0.00%
o2_e,EX_o2_e,21.83,0,0.00%


In [8]:
ylip.reactions.EX_glc_e.lower_bound=-10
ylip.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
glc__D_e,EX_glc_e,10,6,100.00%
h_e,EX_h_e,0.3853,0,0.00%
nh4_e,EX_nh4_e,7.772,0,0.00%
o2_e,EX_o2_e,14.99,0,0.00%
pi_e,EX_pi_e,0.3853,0,0.00%
so4_e,EX_so4_e,0.07735,0,0.00%
Metabolite,Reaction,Flux,C-Number,C-Flux
biomass_c,EX_biom,-1.14,0,0.00%
co2_e,EX_co2_e,-16.79,1,100.00%
h2o_e,EX_h2o_e,-32.53,0,0.00%


In [9]:
sce.reactions.EX_glc__D_e.lower_bound=-10
sce.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.0001838,0,0.00%
cl_e,EX_cl_e,0.001093,0,0.00%
cu2_e,EX_cu2_e,0.0005583,0,0.00%
fe2_e,EX_fe2_e,2.66E-05,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
h_e,EX_h_e,0.4285,0,0.00%
k_e,EX_k_e,0.003075,0,0.00%
mg2_e,EX_mg2_e,0.001053,0,0.00%
mn2_e,EX_mn2_e,0.002313,0,0.00%
na1_e,EX_na1_e,0.003363,0,0.00%


In [10]:
eco.reactions.EX_glc__D_e.lower_bound=-10
eco.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
ca2_e,EX_ca2_e,0.005113,0,0.00%
cl_e,EX_cl_e,0.005113,0,0.00%
cobalt2_e,EX_cobalt2_e,2.456E-05,0,0.00%
cu2_e,EX_cu2_e,0.0006965,0,0.00%
fe2_e,EX_fe2_e,0.01578,0,0.00%
glc__D_e,EX_glc__D_e,10,6,100.00%
k_e,EX_k_e,0.1918,0,0.00%
mg2_e,EX_mg2_e,0.008522,0,0.00%
mn2_e,EX_mn2_e,0.0006788,0,0.00%
mobd_e,EX_mobd_e,0.0001267,0,0.00%


#### generate flux solutions for each model. 

In [11]:
lst_glc_soln = pfba(lst)#.optimize()
rto_glc_soln = pfba(rto)#.optimize()
ylip_glc_soln = pfba(ylip)#.optimize()
sce_glc_soln = pfba(sce)#.optimize()
# eco_glc_soln = pfba(eco)#.optimize()


function to find reactions producing/consuming specified metabolites. 

In [12]:

def get_reaction_contribution_to_metabolite_production(metabolite_ids,solution,model=lst):
    ''' Main routine that returns reactions that produce and consume a metabolite
    
    Args:
        metabolite_ids (cobra.core.model.Model.metabolites.id):
            Metabolite ID to find.
        solution (cobra.flux_analysis.pfba):
            pfba solution.
        model (cobra.core.model.Model):
            Genome-scael metabolic model to find metabolites and reactions.
    '''
    
    
    # Step 1: Calculate the total production.
    total_production = 0
    producing_reactions = {}
    total_consumption = 0
    consuming_reactions = {}
    
    for met_id in metabolite_ids:
        metabolite = model.metabolites.get_by_id(met_id)
        
        for rxn in metabolite.reactions:
            flux = solution.fluxes[rxn.id]
            
            if (rxn.id!='NADPHtru'):
                # Check if the reaction produces the metabolite.
                if (flux < 0 and (rxn.get_coefficient(met_id) < 0)) | (flux > 0 and (rxn.get_coefficient(met_id) > 0)):
                    production = abs(rxn.get_coefficient(met_id)) * flux
                    total_production += production

                    # Store the individual production for this reaction.
                    producing_reactions[rxn.id] = production

                # Check if the reaction produces the metabolite.
                if (flux > 0 and (rxn.get_coefficient(met_id) < 0)) | (flux < 0 and (rxn.get_coefficient(met_id) > 0)):
                    consumption = abs(rxn.get_coefficient(met_id)) * abs(flux)
                    total_consumption += consumption

                    # Store the individual production for this reaction.
                    consuming_reactions[rxn.id] = consumption

    # Step 2: Calculate the percentage contribution.
    reaction_contributions_produce = {}
    for rxn_id, production in producing_reactions.items():
        reaction_contributions_produce[rxn_id] = abs(production) / total_production * 100
    
    
    # Step 2: Calculate the percentage contribution.
    reaction_contributions_consume = {}
    for rxn_id, consumption in consuming_reactions.items():
        reaction_contributions_consume[rxn_id] = abs(consumption) / total_consumption * 100


    return reaction_contributions_produce,reaction_contributions_consume


# Step one. Verify the subcellular location of the glyoxylate cycle and related proteins. 

Using the Bologna Unified Subcellular Component Annotator ([BUSCA](https://busca.biocomp.unibo.it/)) for predictions. For details see: Savojardo, C, Martelli, PL, Fariselli, P, Profiti, G, Casadio, R (2018) BUSCA: an integrative web server to predict subcellular localization of proteins. [Nucleid Acids Research, 46(W1), W459-W466.](https://academic.oup.com/nar/article/46/W1/W459/4990035?login=true)



#### Check current genes and locations in the model.  

#### ICDH

In [15]:
lst.reactions.ICDHyp

Reaction identifier,ICDHyp
Name,Isocitrate dehydrogenase NADP
Memory address,0x17cb03e50
Stoichiometry,icit_x + nadp_x --> akg_x + co2_x + nadph_x Isocitrate + Nicotinamide adenine dinucleotide phosphate --> 2-Oxoglutarate + CO2 + Nicotinamide adenine dinucleotide phosphate - reduced
GPR,Lipst1_1_85639
Lower bound,0.0
Upper bound,1000.0


#### location result check. 


- BUSCA predicts cytoplasm. 
- Deeploc predicts peroxisomal targeting signal. 

move reaction to both compartments. 

In [16]:
r = lst.reactions.ICDHyp.copy()
r.id = "ICDHy"

r.add_metabolites({lst.metabolites.nadp_c: -1,
                  lst.metabolites.nadp_x: 1,
                  lst.metabolites.nadph_x: -1,
                  lst.metabolites.nadph_c: 1,
                  lst.metabolites.icit_x: 1,
                  lst.metabolites.icit_c: -1,
                  lst.metabolites.akg_c: 1,
                  lst.metabolites.akg_x: -1,
                  lst.metabolites.co2_c: 1,
                  lst.metabolites.co2_x: -1                  
                  
                  })
lst.add_reactions([r])

In [11]:
# r = lst.reactions.ICDHyp.copy()
# r.id = "ICDHym"

# r.add_metabolites({lst.metabolites.nadp_m: -1,
#                   lst.metabolites.nadp_x: 1,
#                   lst.metabolites.nadph_x: -1,
#                   lst.metabolites.nadph_m: 1,
#                   lst.metabolites.icit_x: 1,
#                   lst.metabolites.icit_m: -1,
#                   lst.metabolites.akg_m: 1,
#                   lst.metabolites.akg_x: -1,
#                   lst.metabolites.co2_m: 1,
#                   lst.metabolites.co2_x: -1                  
                  
#                   })
# lst.add_reactions([r])

In [17]:
lst.reactions.ICDHxm

Reaction identifier,ICDHxm
Name,Isocitrate dehydrogenase NAD
Memory address,0x17cc6c590
Stoichiometry,icit_m + nad_m --> akg_m + co2_m + nadh_m Isocitrate + Nicotinamide adenine dinucleotide --> 2-Oxoglutarate + CO2 CO2 + Nicotinamide adenine dinucleotide - reduced
GPR,Lipst1_1_70496 or Lipst1_1_4980
Lower bound,0.0
Upper bound,1000.0


#### predictions. 
- Lipst1_1_70496:  mitochondrial.
- Lipst1_1_4980: mitochondrial.



https://busca.biocomp.unibo.it/



#### Citrate synthase. 

In [18]:
lst.reactions.CSm

Reaction identifier,CSm
Name,Citrate synthase
Memory address,0x17c928cd0
Stoichiometry,accoa_m + h2o_m + oaa_m --> cit_m + coa_m + h_m Acetyl-CoA + H2O H2O + Oxaloacetate --> Citrate + Coenzyme A + H+
GPR,Lipst1_1_92627 or Lipst1_1_47067
Lower bound,0.0
Upper bound,1000.0


In [19]:
lst.reactions.CSp

Reaction identifier,CSp
Name,Citrate synthase
Memory address,0x17c8fcf10
Stoichiometry,accoa_x + h2o_x + oaa_x --> cit_x + coa_x + h_x Acetyl-CoA + H2O H2O + Oxaloacetate --> Citrate + Coenzyme A + H+
GPR,Lipst1_1_68065 or Lipst1_1_47067
Lower bound,0.0
Upper bound,1000.0


location check. 


protein: 68065 
- bucas: cytoplasm.
- Deeploc: peroxisome.

protein: 47067 

- bucas: mitochondrial membrane. 
- Deeploc - cytoplasm with probability of mito.  

protein: 92627 

- bucas: mitochondrial membrane. 
- Deeploc - cytoplasm with some probability of mito. 


modify CSp location and GPR.

#### added reaction #1. 

In [20]:
## not coppying reaction, moving location. 
r = lst.reactions.CSp.copy()
r.id = 'CS'
r.add_metabolites({lst.metabolites.accoa_x: 1,
                  lst.metabolites.accoa_c: -1,
                  lst.metabolites.h2o_x: 1,
                  lst.metabolites.h2o_c: -1,
                  lst.metabolites.oaa_x: 1,
                  lst.metabolites.oaa_c: -1,
                  lst.metabolites.cit_x: -1,
                  lst.metabolites.cit_c: 1,
                  lst.metabolites.coa_x: -1,
                  lst.metabolites.coa_c: 1,
                  lst.metabolites.h_x: -1, 
                  lst.metabolites.h_c: 1
                  
                  })

r.gene_reaction_rule = 'Lipst1_1_68065'
lst.add_reactions([r])

In [21]:
r = lst.reactions.CSp
r.gene_reaction_rule = 'Lipst1_1_68065'


In [22]:
r

Reaction identifier,CSp
Name,Citrate synthase
Memory address,0x17c8fcf10
Stoichiometry,accoa_x + h2o_x + oaa_x --> cit_x + coa_x + h_x Acetyl-CoA + H2O H2O + Oxaloacetate --> Citrate + Coenzyme A + H+
GPR,Lipst1_1_68065
Lower bound,0.0
Upper bound,1000.0


#### exmaine malate dehydrogenase. 



In [23]:
lst.reactions.MDHp

Reaction identifier,MDHp
Name,Malate dehydrogenase peroxisomal
Memory address,0x17ccc6210
Stoichiometry,mal__L_x + nad_x <=> h_x + nadh_x + oaa_x (S)-Malate + NAD+ <=> H+ + NADH + Oxaloacetate
GPR,Lipst1_1_3988 or Lipst1_1_5229
Lower bound,-1000.0
Upper bound,1000.0


In [24]:
lst.reactions.MDHm

Reaction identifier,MDHm
Name,Malate dehydrogenase mitochondrial
Memory address,0x17cb78750
Stoichiometry,mal__L_m + nad_m <=> h_m + nadh_m + oaa_m L-Malate + Nicotinamide adenine dinucleotide <=> H+ + Nicotinamide adenine dinucleotide - reduced + Oxaloacetate
GPR,Lipst1_1_5229
Lower bound,-1000.0
Upper bound,1000.0


#### examine malate dehydrogenase.


Lipst1_1_3988:
- BUsCA - secreted - (note, this is unlikely to be secreted in reality). 
- DeepLoc: leaving in cytoplasm location. 

Lipst1_1_5229:
- BUSCA cytoplasm.
- DeepLoc: Lysome/Vaculoue. 

unclear predictions.
setting to 'or' GPR.

####  MDHp to be in cytoplasm. 

In [25]:
lst.reactions.MDHm.gene_reaction_rule = 'Lipst1_1_5229 or Lipst1_1_3988'

In [26]:
lst.reactions.MDHp.gene_reaction_rule = 'Lipst1_1_5229 or Lipst1_1_3988'

In [27]:
# move MDHp to cytoplasm.
r = lst.reactions.MDHp.copy()
# r = lst.reactions.MDHp

r.id = 'MDH'
r.name = 'Malate dehydrogenase'
r.add_metabolites({lst.metabolites.mal__L_x: 1,
                  lst.metabolites.mal__L_c: -1,
                  lst.metabolites.nad_x: 1,
                  lst.metabolites.nad_c: -1,
                  lst.metabolites.h_x: -1,
                  lst.metabolites.h_c: 1,
                  lst.metabolites.nadh_c: 1,
                  lst.metabolites.nadh_x: -1,
                  lst.metabolites.oaa_c: 1,
                  lst.metabolites.oaa_x: -1,
                                     
                  })

lst.add_reactions([r])

# r

#### check Malate synthase.



In [28]:
lst.reactions.MALSp

Reaction identifier,MALSp
Name,Malate synthase
Memory address,0x17e431a50
Stoichiometry,accoa_x + glx_x + h2o_x --> coa_x + h_x + mal__L_x Acetyl-CoA + Glyoxylate + H2O H2O --> Coenzyme A + H+ + (S)-Malate
GPR,Lipst1_1_73436
Lower bound,0.0
Upper bound,1000.0


Lipst1_1_73436. Malate synthase. 
- BUSCA: cytoplasm.
- Deeploc: peroxisome. 

In [29]:
r = lst.reactions.MALSp.copy()

r.id = 'MALS'
r.name = 'Malate synthase'
r.add_metabolites({lst.metabolites.accoa_x: 1,
                  lst.metabolites.accoa_c: -1,
                  lst.metabolites.glx_x: 1,
                  lst.metabolites.glx_c: -1,
                  lst.metabolites.h2o_x: 1,
                  lst.metabolites.h2o_c: -1,
                  lst.metabolites.coa_c: 1,
                  lst.metabolites.coa_x: -1,
                  lst.metabolites.h_c: 1,
                  lst.metabolites.h_x: -1,
                  lst.metabolites.mal__L_c: 1,
                  lst.metabolites.mal__L_x: -1})
lst.add_reactions([r])
r                          

Reaction identifier,MALS
Name,Malate synthase
Memory address,0x17e55eb10
Stoichiometry,accoa_c + glx_c + h2o_c --> coa_c + h_c + mal__L_c Acetyl-CoA + Glyoxylate + H2O H2O --> Coenzyme A + H+ + L-Malate
GPR,Lipst1_1_73436
Lower bound,0.0
Upper bound,1000.0


#### incomplete glyoxlyate cycle in mitochondria without MALSm. Add to model.



In [105]:
r = lst.reactions.MALSp.copy()

r.id = 'MALSm'
r.name = 'Malate synthase'
r.add_metabolites({lst.metabolites.accoa_x: 1,
                  lst.metabolites.accoa_m: -1,
                  lst.metabolites.glx_x: 1,
                  lst.metabolites.glx_m: -1,
                  lst.metabolites.h2o_x: 1,
                  lst.metabolites.h2o_m: -1,
                  lst.metabolites.coa_m: 1,
                  lst.metabolites.coa_x: -1,
                  lst.metabolites.h_m: 1,
                  lst.metabolites.h_x: -1,
                  lst.metabolites.mal__L_m: 1,
                  lst.metabolites.mal__L_x: -1})
lst.add_reactions([r])
r                          

Reaction identifier,MALSm
Name,Malate synthase
Memory address,0x28bf53010
Stoichiometry,accoa_m + glx_m + h2o_m --> coa_m + h_m + mal__L_m Acetyl-CoA + Glyoxylate + H2O H2O --> Coenzyme A + H+ + L-Malate
GPR,Lipst1_1_73436
Lower bound,0.0
Upper bound,1000.0


#### Isocitrate lyase. 

In [30]:
lst.reactions.ICLp

Reaction identifier,ICLp
Name,Isocitrate lyase
Memory address,0x17ce85e90
Stoichiometry,icit_x --> glx_x + succ_x Isocitrate --> Glyoxylate + Succinate
GPR,Lipst1_1_112377 or Lipst1_1_75138
Lower bound,0.0
Upper bound,1000.0


#### Isocitrate lyase. 

- Lipst1_1_112377 
    - BUSCA: cytoplasm
    - Deeploc: cytoplasm with peroxisome signalling. 
- Lipst1_1_75138 
    - mitochondria

In [31]:
r = lst.reactions.ICLp.copy()
r.id = 'ICL'
r.name = 'Isocitrate lyase'
r.gene_reaction_rule = 'Lipst1_1_112377'
r.add_metabolites({lst.metabolites.icit_c: -1,
                  lst.metabolites.icit_x: 1,
                  lst.metabolites.glx_x: -1,
                  lst.metabolites.glx_c: 1,
                  lst.metabolites.succ_x: -1,
                  lst.metabolites.succ_c: 1,
                                     
                  })

r
lst.add_reactions([r])

In [32]:
r = lst.reactions.ICLp.copy()
r.id = 'ICLm'
r.name = 'Isocitrate lyase mitochondria'
r.gene_reaction_rule = 'Lipst1_1_75138'
r.add_metabolites({lst.metabolites.icit_m: -1,
                  lst.metabolites.icit_x: 1,
                  lst.metabolites.glx_x: -1,
                  lst.metabolites.glx_m: 1,
                  lst.metabolites.succ_x: -1,
                  lst.metabolites.succ_m: 1,
                                     
                  })
lst.add_reactions([r])
r

Reaction identifier,ICLm
Name,Isocitrate lyase mitochondria
Memory address,0x28be6d7d0
Stoichiometry,icit_m --> glx_m + succ_m Isocitrate --> Glyoxylate + Succinate
GPR,Lipst1_1_75138
Lower bound,0.0
Upper bound,1000.0


#### malic enzyme. 

In [33]:
lst.reactions.ME1m

Reaction identifier,ME1m
Name,Malic enzyme NAD mitochondrial
Memory address,0x17e4677d0
Stoichiometry,mal__L_m + nad_m --> co2_m + nadh_m + pyr_m L-Malate + Nicotinamide adenine dinucleotide --> CO2 CO2 + Nicotinamide adenine dinucleotide - reduced + Pyruvate
GPR,Lipst1_1_72728
Lower bound,0.0
Upper bound,1000.0


- Lipst1_1_72728 
    - BUSCA cytoplasm. 
    - Deeploc: cytoplasm and peroxisome targetting. 

In [34]:
r = lst.reactions.ME1m.copy()
r.id = 'ME1'
r.name = 'Malic enzyme'
r.gene_reaction_rule = 'Lipst1_1_72728'
r.add_metabolites({lst.metabolites.mal__L_c: -1,
                  lst.metabolites.mal__L_m: 1,
                  lst.metabolites.nad_c: -1,
                  lst.metabolites.nad_m: 1,
                  lst.metabolites.co2_m: -1,
                  lst.metabolites.co2_c: 1,
                  lst.metabolites.nadh_m: -1,
                  lst.metabolites.nadh_c: 1,
                  lst.metabolites.pyr_m: -1,
                  lst.metabolites.pyr_c: 1})

lst.add_reactions([r])


### pryuvate is not in peroxisome. 

m = lst.metabolites.pyr_c.copy()
m.id = 'pyr_x'
lst.add_metabolites([m])


r = lst.reactions.ME1m.copy()
r.id = 'ME1p'
r.name = 'Malic enzyme'
r.gene_reaction_rule = 'Lipst1_1_72728'
r.add_metabolites({lst.metabolites.mal__L_x: -1,
                  lst.metabolites.mal__L_m: 1,
                  lst.metabolites.nad_x: -1,
                  lst.metabolites.nad_m: 1,
                  lst.metabolites.co2_m: -1,
                  lst.metabolites.co2_x: 1,
                  lst.metabolites.nadh_m: -1,
                  lst.metabolites.nadh_x: 1,
                  lst.metabolites.pyr_m: -1,
                  lst.metabolites.pyr_x: 1})

lst.add_reactions([r])

add transporter for pyruvate in peroxisome. 

[reaction](http://bigg.ucsd.edu/universal/reactions/PYRt_x)

In [35]:
r = lst.reactions.ME1.copy()
r.id = 'PYRt_x'
r.name = 'Pyruvate peroxisome transporter'
r.gene_reaction_rule = ''
r.add_metabolites({lst.metabolites.mal__L_c: 1,
                  lst.metabolites.nad_c: 1,
                  lst.metabolites.co2_c: -1,
                  lst.metabolites.nadh_c: -1,
                  lst.metabolites.pyr_c: -2,
#                   lst.metabolites.pyr_c: -1,

                  lst.metabolites.pyr_x: 1})
r.bounds = -1000,1000

lst.add_reactions([r])

In [36]:
r

Reaction identifier,PYRt_x
Name,Pyruvate peroxisome transporter
Memory address,0x28be77b50
Stoichiometry,pyr_c <=> pyr_x Pyruvate <=> Pyruvate
GPR,
Lower bound,-1000
Upper bound,1000


#### oxoalacetate decarboyxlase & PPCK.

In [37]:
lst.reactions.OAADCm

Reaction identifier,OAADCm
Name,Oxaloacetate decarboxylase mitochondrial
Memory address,0x17e94c3d0
Stoichiometry,h_m + oaa_m --> co2_m + pyr_m H+ + Oxaloacetate --> CO2 CO2 + Pyruvate
GPR,Lipst1_1_72728
Lower bound,0.0
Upper bound,1000.0


In [38]:
lst.reactions.PPCK

Reaction identifier,PPCK
Name,Phosphoenolpyruvate carboxykinase
Memory address,0x17cdef950
Stoichiometry,atp_c + oaa_c --> adp_c + co2_c + pep_c ATP C10H12N5O13P3 + Oxaloacetate --> ADP C10H12N5O10P2 + CO2 CO2 + Phosphoenolpyruvate
GPR,Lipst1_1_934
Lower bound,0.0
Upper bound,1000.0


- OAACDm
    - BUSCA: cyotoplasm
    - Deeploc: cytoplasm with Peroxisome tag.
- PPCK
    - cytoplasm
    - Deeploc: cytoplasm with Peroxisome tag. -however no pep_x, no adp_x in model. 


In [39]:
r = lst.reactions.OAADCm.copy()
r.id = 'OAADC'
r.name = 'Oxaloacetate decarboxylase'
r.add_metabolites({lst.metabolites.oaa_c: -1,
                  lst.metabolites.oaa_m: 1,
                  lst.metabolites.h_c: -1,
                  lst.metabolites.h_m: 1,
                  lst.metabolites.co2_m: -1,
                  lst.metabolites.co2_c: 1,
                  lst.metabolites.pyr_m: -1,
                  lst.metabolites.pyr_c: 1})

lst.add_reactions([r])

r = lst.reactions.OAADCm.copy()
r.id = 'OAADCp'
r.name = 'Oxaloacetate decarboxylase'
r.add_metabolites({lst.metabolites.oaa_x: -1,
                  lst.metabolites.oaa_m: 1,
                  lst.metabolites.h_x: -1,
                  lst.metabolites.h_m: 1,
                  lst.metabolites.co2_m: -1,
                  lst.metabolites.co2_x: 1,
                  lst.metabolites.pyr_m: -1,
                  lst.metabolites.pyr_x: 1})
lst.add_reactions([r])

r

Ignoring reaction 'OAADC' since it already exists.


Reaction identifier,OAADCp
Name,Oxaloacetate decarboxylase
Memory address,0x28c309c90
Stoichiometry,h_x + oaa_x --> co2_x + pyr_x H+ + Oxaloacetate --> CO2 + Pyruvate
GPR,Lipst1_1_72728
Lower bound,0.0
Upper bound,1000.0


In [40]:
# r = lst.reactions.OAADC.copy()
# r.id = 'OAAtm'
# r.name = 'Oxaloacetate decarboxylase'
# r.add_metabolites({
#                   lst.metabolites.oaa_m: 1,
#                   lst.metabolites.h_c: 1,
#                   lst.metabolites.co2_c: -1,
#                   lst.metabolites.pyr_c: -1})
# r.bounds = -1000,1000
# lst.add_reactions([r])

In [41]:
lst.reactions.PPCK

Reaction identifier,PPCK
Name,Phosphoenolpyruvate carboxykinase
Memory address,0x17cdef950
Stoichiometry,atp_c + oaa_c --> adp_c + co2_c + pep_c ATP C10H12N5O13P3 + Oxaloacetate --> ADP C10H12N5O10P2 + CO2 CO2 + Phosphoenolpyruvate
GPR,Lipst1_1_934
Lower bound,0.0
Upper bound,1000.0


In [42]:

m = lst.metabolites.adp_c.copy()
m.id = 'adp_x'
lst.add_metabolites([m])

# m = lst.metabolites.pep_c.copy()
# m.id = 'pep_x'
# lst.add_metabolites([m])

# r = lst.reactions.PPCK.copy()
# r.id = 'PPCKp'
# r.name = 'Phosphoenolpyruvate carboxykinase peroxisome'
# r.add_metabolites({lst.metabolites.oaa_x: -1,
#                   lst.metabolites.oaa_c: 1,
#                   lst.metabolites.atp_x: -1,
#                   lst.metabolites.atp_c: 1,
#                   lst.metabolites.adp_c: -1,
#                   lst.metabolites.adp_x: 1,
#                   lst.metabolites.co2_c: -1,
#                   lst.metabolites.co2_x: 1,
#                   lst.metabolites.pep_c: -1,
#                   lst.metabolites.pep_x: 1})
# r

<!-- add adp_x transporter and metabolite.

[bigg reaction](http://bigg.ucsd.edu/universal/reactions/ADPtx)

add PYRt_x  -->

In [43]:
r = lst.reactions.ME1.copy()
r.id = 'ADPtx'
r.name = 'ADP peroxisome transporter'
r.gene_reaction_rule = ''
r.add_metabolites({lst.metabolites.mal__L_c: 1,
                  lst.metabolites.nad_c: 1,
                  lst.metabolites.co2_c: -1,
                  lst.metabolites.nadh_c: -1,
                  lst.metabolites.pyr_c: -1,
                  lst.metabolites.adp_c: -1,

                  lst.metabolites.adp_x: 1})
r.bounds = -1000,1000

lst.add_reactions([r])

In [44]:
# lst.add_reactions([r])


In [45]:
lst.optimize()

,fluxes,reduced_costs
ALCD25yi,0.0,1.110223e-16
MTHFCm,0.0,-0.000000e+00
AMPN,0.0,-4.356738e-03
DAGCPTer_RT,0.0,-1.110223e-16
PYRt2,0.0,0.000000e+00
...,...,...
ME1,0.0,-4.356738e-03
ME1p,0.0,-4.356738e-03
PYRt_x,0.0,0.000000e+00
OAADCp,0.0,-4.356738e-03


#### Aspartate transaminase. 

In [46]:
lst.reactions.ASPTAm

Reaction identifier,ASPTAm
Name,Aspartate transaminase
Memory address,0x17cf103d0
Stoichiometry,akg_m + asp__L_m <=> glu__L_m + oaa_m 2-Oxoglutarate + L-Aspartate <=> L-Glutamate + Oxaloacetate
GPR,Lipst1_1_315712 or Lipst1_1_5184
Lower bound,-1000.0
Upper bound,1000.0


In [47]:
lst.reactions.ASPTA

Reaction identifier,ASPTA
Name,Aspartate transaminase
Memory address,0x17cab9d10
Stoichiometry,akg_c + asp__L_c <=> glu__L_c + oaa_c 2-Oxoglutarate + L-Aspartate <=> L-Glutamate + Oxaloacetate
GPR,Lipst1_1_5184
Lower bound,-1000.0
Upper bound,1000.0


>5184
- BUSCA organelle membrane
- Deeploc (cytoplasm)

>315712
- BUSCA mitochondrial membrane. 
- Deeploc mitochondrion.

membrane proteins may be facing either way. However, previous work assumed ASPTA is only [cytolsolic](https://academic.oup.com/femsyr/article/5/6-7/545/528244). 

In [48]:
lst.reactions.ASPTA.gene_reaction_rule = 'Lipst1_1_315712 or Lipst1_1_5184'
lst.reactions.ASPTAm.bounds = 0,0

In [49]:
lst.optimize()

,fluxes,reduced_costs
ALCD25yi,0.0,1.110223e-16
MTHFCm,0.0,-0.000000e+00
AMPN,0.0,-4.356738e-03
DAGCPTer_RT,0.0,-1.110223e-16
PYRt2,0.0,0.000000e+00
...,...,...
ME1,0.0,-4.356738e-03
ME1p,0.0,-4.356738e-03
PYRt_x,0.0,0.000000e+00
OAADCp,0.0,-4.356738e-03


## begin examining cofactor cycles. 


### NADPH start. 

In [46]:
# cobra.io.save_json_model(lst,'test_with_compartments.json')

In [50]:
# pfba(lst).fluxes.to_csv('test_w_compartments_.csv')

In [51]:
lst_glc_soln = pfba(lst)
# lst_glc_soln = lst.optimize()
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 13.40% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 13.40% of the total NADPH production.
Reaction ALDD19x_P contributes 63.63% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reactio

### constrain the feasibility of ALCD reactions to prevent cofactor cycling. 

In [52]:
lst.reactions.ALCD2y

Reaction identifier,ALCD2y
Name,"Alcohol dehydrogenase (ethanol, NADP)"
Memory address,0x17cb2db90
Stoichiometry,etoh_c + nadp_c --> acald_c + h_c + nadph_c Ethanol + Nicotinamide adenine dinucleotide phosphate --> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide phosphate - reduced
GPR,Lipst1_1_177423 or Lipst1_1_121954
Lower bound,0.0
Upper bound,1000.0


In [53]:
lst_glc_soln.fluxes['ALCD2y']

0.0

In [54]:
for rx in lst.metabolites.acald_c.reactions:
    if lst_glc_soln.fluxes[rx.id]!=0:
        print(lst_glc_soln.fluxes[rx.id],rx)

-0.9599360855009768 ACALDtm: acald_m <=> acald_c
0.9599360855009768 PYRDC: h_c + pyr_c --> acald_c + co2_c


there is an nadh = nadph cycle due to the ALCD2x and ALCD2y reactions. check constraints in other models. 

there is no ALCD2y reaction in the yeast8 model.

In [55]:
# sce.reactions.ALCD2y

In [56]:
for r in lst.metabolites.etoh_c.reactions:
    if lst_glc_soln[r.id]!=0:
        print(r.id,lst_glc_soln[r.id],r.subsystem,r.name)

In [57]:
lst.reactions.ALCD2x

Reaction identifier,ALCD2x
Name,Alcohol dehydrogenase (ethanol)
Memory address,0x17ca09550
Stoichiometry,etoh_c + nad_c <=> acald_c + h_c + nadh_c Ethanol + Nicotinamide adenine dinucleotide <=> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,Lipst1_1_99746
Lower bound,-1000.0
Upper bound,1000.0


In [58]:
sce.reactions.ALCD2x

Reaction identifier,ALCD2x
Name,alcohol dehydrogenase (ethanol to acetaldehyde)
Memory address,0x289f205d0
Stoichiometry,etoh_c + nad_c --> acald_c + h_c + nadh_c ethanol + NAD --> acetaldehyde + H+ + NADH
GPR,YMR303C
Lower bound,0.0
Upper bound,1000.0


In [59]:
rto.reactions.ALCD2x

Reaction identifier,ALCD2x
Name,Alcohol dehydrogenase (ethanol)
Memory address,0x17f4b1290
Stoichiometry,etoh_c + nad_c <=> acald_c + h_c + nadh_c Ethanol + Nicotinamide adenine dinucleotide <=> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide - reduced
GPR,15438
Lower bound,-1000.0
Upper bound,1000.0


In [60]:
rto.reactions.ALCD2y

Reaction identifier,ALCD2y
Name,"Alcohol dehydrogenase (ethanol, NADP)"
Memory address,0x17f620ed0
Stoichiometry,etoh_c + nadp_c --> acald_c + h_c + nadph_c Ethanol + Nicotinamide adenine dinucleotide phosphate --> Acetaldehyde + H+ + Nicotinamide adenine dinucleotide phosphate - reduced
GPR,11882 or 12784 or 9774
Lower bound,0.0
Upper bound,1000.0


In [61]:
# sce.reactions.ALCD2y

#### introduced constraint #1.

set bounds on on ALCD2y - blocking reaction. 

In [62]:
# lst.reactions.ALCD2x.lower_bound=-1000
lst.reactions.ALCD2y.bounds=0,0

In [63]:
# lst_glc_soln = lst.optimize()
lst_glc_soln = pfba(lst)
# lst_glc_fva_soln = cobra.flux_analysis.flux_variability_analysis(lst)

In [64]:
lst_glc_soln = pfba(lst)
# lst_glc_soln = lst.optimize()
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 13.40% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction HSDy contributes 63.63% of the total NADPH production.
Reaction G6PDH2r contributes 13.40% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reaction AAS

examine HSDy reaction.

In [65]:
lst.reactions.HSDy

Reaction identifier,HSDy
Name,Homoserine dehydrogenase (NADPH)
Memory address,0x17e474b90
Stoichiometry,hom__L_c + nadp_c <=> aspsa_c + h_c + nadph_c L-Homoserine + Nicotinamide adenine dinucleotide phosphate <=> L-Aspartate 4-semialdehyde + H+ + Nicotinamide adenine dinucleotide phosphate - reduced
GPR,Lipst1_1_3431
Lower bound,-1000.0
Upper bound,1000.0


In [66]:
for r in lst.metabolites.aspsa_c.reactions:
    if lst_glc_soln[r.id]!=0:
        print(r.id,lst_glc_soln[r.id],r.subsystem,r.name)

ASAD -0.44868728406647834 Threonine and Lysine Metabolism Aspartate-semialdehyde dehydrogenase
HSDy 10.23652185765077 Threonine and Lysine Metabolism Homoserine dehydrogenase (NADPH)
HSDxi 10.685209141717248 Glycine and Serine Metabolism Homoserine dehydrogenase  NADH   irreversible


there is a nadph = nadh cycle through HSDy and HSDxi. Examine the constraints in ohter models. 

In [67]:
rto.reactions.HSDy

Reaction identifier,HSDy
Name,Homoserine dehydrogenase (NADPH)
Memory address,0x17f988fd0
Stoichiometry,hom__L_c + nadp_c <=> aspsa_c + h_c + nadph_c L-Homoserine + Nicotinamide adenine dinucleotide phosphate <=> L-Aspartate 4-semialdehyde + H+ + Nicotinamide adenine dinucleotide phosphate - reduced
GPR,12080 or 16738
Lower bound,-1000.0
Upper bound,1000.0


In [68]:
sce.reactions.HSDy

Reaction identifier,HSDy
Name,homoserine dehydrogenase (NADP)
Memory address,0x28a012790
Stoichiometry,aspsa_c + h_c + nadph_c --> hom__L_c + nadp_c L-aspartate 4-semialdehyde + H+ + NADPH --> L-homoserine + NADP(+)
GPR,YJR139C
Lower bound,0.0
Upper bound,1000.0


In [69]:
sce.reactions.HSDxi

Reaction identifier,HSDxi
Name,homoserine dehydrogenase (NADH)
Memory address,0x28a012090
Stoichiometry,aspsa_c + h_c + nadh_c --> hom__L_c + nad_c L-aspartate 4-semialdehyde + H+ + NADH --> L-homoserine + NAD
GPR,YJR139C
Lower bound,0.0
Upper bound,1000.0


In [70]:
rto.reactions.HSDxi

Reaction identifier,HSDxi
Name,Homoserine dehydrogenase NADH irreversible
Memory address,0x17f727b50
Stoichiometry,aspsa_c + h_c + nadh_c --> hom__L_c + nad_c L-Aspartate 4-semialdehyde + H+ + Nicotinamide adenine dinucleotide - reduced --> L-Homoserine + Nicotinamide adenine dinucleotide
GPR,12080 or 16738
Lower bound,0.0
Upper bound,1000.0


#### introduced constraint #2.

HSDy is irrevisible in sce. following sce convention. 

In [71]:
lst.reactions.HSDy.upper_bound = 0

In [72]:
lst_glc_soln = pfba(lst)

nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 13.40% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 13.40% of the total NADPH production.
Reaction ALDD19x_P contributes 63.63% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reactio

Examine ALDD19x_P.

In [73]:
lst.reactions.get_by_id('ALDD19x_P')

Reaction identifier,ALDD19x_P
Name,"Aldehyde dehydrogenase (phenylacetaldehyde, NADP)"
Memory address,0x17e882c90
Stoichiometry,h2o_c + nadp_c + pacald_c --> 2.0 h_c + nadph_c + pac_c H2O H2O + Nicotinamide adenine dinucleotide phosphate + Phenylacetaldehyde --> 2.0 H+ + Nicotinamide adenine dinucleotide phosphate - reduced + Phenylacetic acid
GPR,Lipst1_1_314464 or Lipst1_1_291711 or Lipst1_1_767
Lower bound,0.0
Upper bound,1000.0


In [74]:
for r in lst.metabolites.pac_c.reactions:
    if lst_glc_soln[r.id]!=0:
        print(r.id,lst_glc_soln[r.id],r.subsystem,r.name)

ALDD19x_P 10.236521857650354 Phenylalanine Metabolism Aldehyde dehydrogenase (phenylacetaldehyde, NADP)
ALDD19xr -10.236521857650354 Alternate Carbon Metabolism Aldehyde dehydrogenase (phenylacetaldehyde, NAD)


there is a nadh = nadph cycle due to reactions ALDD19x_P and ALDD19xr. examine reactions in other models. 

there is no ALDD19x_P reaction in the yeast8 model.

In [75]:
# sce.reactions.ALDD19x_P

look for a cofactor futile cycle.

In [76]:
sce.reactions.ALDD19xr

Reaction identifier,ALDD19xr
Name,"aldehyde dehydrogenase (phenylacetaldehyde, NAD)"
Memory address,0x289f4ae90
Stoichiometry,h2o_c + nad_c + pacald_c --> 2.0 h_c + nadh_c + pac_c H2O + NAD + phenylacetaldehyde --> 2.0 H+ + NADH + phenylacetic acid
GPR,YMR169C or YMR170C
Lower bound,0.0
Upper bound,1000.0


#### introduced constraint #3 - set bounds on ALDD19x_P to 0, blocking reaction.

In [77]:
lst.reactions.get_by_id('ALDD19x_P').bounds=0,0
# lst.reactions.get_by_id('ALDD19xr').bounds=0,0

In [78]:
lst_glc_soln = pfba(lst)#.optimize()

In [79]:
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 45.21% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 45.21% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reaction AASAD1 contributes 0.95% of the total NADPH consumption.
Reaction S

Compare the sce and ylip.

In [80]:
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_p"],sce_glc_soln,sce)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction MTHFD contributes 4.03% of the total NADPH production.
Reaction ICDHyr contributes 3.37% of the total NADPH production.
Reaction G6PDH2r contributes 36.05% of the total NADPH production.
Reaction r_0234 contributes 0.30% of the total NADPH production.
Reaction ALDD2y contributes 10.81% of the total NADPH production.
Reaction GND contributes 36.05% of the total NADPH production.
Reaction SSALy contributes 0.00% of the total NADPH production.
Reaction r_0771 contributes 0.03% of the total NADPH production.
Reaction PPND2 contributes 0.94% of the total NADPH production.
Reaction ICDHym contributes 8.43% of the total NADPH production.


Reaction r_0015 contributes 0.01% of the total NADPH consumption.
Reaction ASAD contributes 2.24% of the total NADPH consumption.
Reaction SQLS contributes 0.30% of the total NADPH consumption.
Reaction HMGCOAR contributes 3.57% of the total NADPH consumption.
Reaction r_2140 contributes 14.10% of the total NADPH consumption.
Reaction C14STR contri

In [81]:
lst_glc_soln = pfba(lst)
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", ],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 45.21% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 45.21% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reaction AASAD1 contributes 0.95% of the total NADPH consumption.
Reaction S

In [83]:
# cobra.io.save_json_model(lst,'test_with_buca_w3_firstconstraints.json')
# pfba(lst).fluxes.to_csv('test_w_busca_w3_firstconstraints.csv')

#### Several transport reactions were not functioning. 

#### introduced constraint.

In [84]:
lst.reactions.ACRNtm.bounds = -1000,1000


In [85]:
lst_glc_soln = pfba(lst)
# lst_glc_soln = lst.optimize()
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 45.21% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 45.21% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reaction AASAD1 contributes 0.95% of the total NADPH consumption.
Reaction S

In [86]:
lst_glc_soln = pfba(lst)
# lst_glc_soln = lst.optimize()
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 45.21% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 45.21% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reaction AASAD1 contributes 0.95% of the total NADPH consumption.
Reaction S

In [87]:
lst_glc_soln['GND']

7.2025618459514815

In [88]:
lst_glc_soln['PGI']

1.7523921653489234

In [89]:

lst_glc_soln['FBA']

3.279276568507006

In [90]:
lst_glc_soln = pfba(lst)
# lst_glc_soln = lst.optimize()
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],lst_glc_soln)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

Reaction GND contributes 45.21% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction G6PDH2r contributes 45.21% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction TRDR contributes 0.71% of the total NADPH consumption.
Reaction SHK3Dr contributes 1.23% of the total NADPH consumption.
Reaction FAS180COA contributes 4.19% of the total NADPH consumption.
Reaction FAS140COA contributes 7.23% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction FAS160COA contributes 7.23% of the total NADPH consumption.
Reaction FAS120COA contributes 7.23% of the total NADPH consumption.
Reaction AASAD1 contributes 0.95% of the total NADPH consumption.
Reaction S

In [88]:
# lst.reactions.ACS.bounds = -1000,1000
# # lst.reactions.ACS.bounds = 0,0
# # # lst.reactions.ACSm.bounds = 0,0
# # # # lst.reactions.ACSm.bounds = -1000,1000

# # # lst.reactions.ACRNtm.bounds = -1000,1000
# # # # lst.reactions.XYLK.bounds = -1000,1000
# lst.reactions.ACt2r.lower_bound =-1000
# lst.reactions.ACtr.lower_bound =-1000
# # lst.reactions.ACt2r.lower_bound = 0
# # lst.reactions.ACtr.lower_bound = 0

# lst.reactions.ACtm.bounds = -1000,1000
# lst.reactions.HMGCOAtm.bounds = -1000,1000
# lst.reactions.ACtm.bounds = 0,0
# lst.reactions.HMGCOAtm.bounds = 0,0


soln = pfba(lst)#optimize()
print(soln['FBA'])
print(soln['GND'])

# Get the percentage contributions
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadph_c", "nadph_m", "nadph_x"],soln,lst)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():

    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADPH consumption.")

0.33373398076257205
0.7140335796182373
Reaction G6PDH2r contributes 45.21% of the total NADPH production.
Reaction MTHFD contributes 2.42% of the total NADPH production.
Reaction GND contributes 45.21% of the total NADPH production.
Reaction FTHFDH contributes 0.75% of the total NADPH production.
Reaction PPND2 contributes 0.44% of the total NADPH production.
Reaction ALDD2ym contributes 5.97% of the total NADPH production.


Reaction DPR contributes 0.00% of the total NADPH consumption.
Reaction SULR contributes 1.90% of the total NADPH consumption.
Reaction G5SD contributes 1.18% of the total NADPH consumption.
Reaction AASAD1 contributes 0.95% of the total NADPH consumption.
Reaction KYN3OX contributes 0.01% of the total NADPH consumption.
Reaction HMGCOAR contributes 0.90% of the total NADPH consumption.
Reaction FAS100COA contributes 7.23% of the total NADPH consumption.
Reaction GLUDy contributes 28.53% of the total NADPH consumption.
Reaction APRAUR contributes 0.01% of the tota

In [91]:
rxn_list = []

for r in lst.reactions:
    sub = r.subsystem
    if sub =='Glycolysis/Gluconeogenesis':
        rxn_list.append(r)

sorted_reactions = sorted(rxn_list, key=lambda reaction: reaction.id)

for r in sorted_reactions:
    if lst_glc_soln[r.id]!=0:
        print(r.id,lst_glc_soln[r.id],r.subsystem,r.name,'\n',r)

ENO 11.92478591854981 Glycolysis/Gluconeogenesis Enolase 
 ENO: 2pg_c <=> h2o_c + pep_c
FBA 3.279276568507006 Glycolysis/Gluconeogenesis Fructose-bisphosphate aldolase 
 FBA: fdp_c <=> dhap_c + g3p_c
GAPD 13.298208262763724 Glycolysis/Gluconeogenesis Glyceraldehyde-3-phosphate dehydrogenase 
 GAPD: g3p_c + nad_c + pi_c <=> 13dpg_c + h_c + nadh_c
HEX1 10.0 Glycolysis/Gluconeogenesis Hexokinase (D-glucose:ATP) 
 HEX1: atp_c + glc__D_c --> adp_c + g6p_c + h_c
PDHm 7.569548557308286 Glycolysis/Gluconeogenesis Pyruvate dehydrogenase 
 PDHm: coa_m + nad_m + pyr_m --> accoa_m + co2_m + nadh_m
PFK 3.279276568507006 Glycolysis/Gluconeogenesis Phosphofructokinase 
 PFK: atp_c + f6p_c --> adp_c + fdp_c + h_c
PGI 1.7523921653489234 Glycolysis/Gluconeogenesis Glucose-6-phosphate isomerase 
 PGI: g6p_c <=> f6p_c
PGK -13.298208262763724 Glycolysis/Gluconeogenesis Phosphoglycerate kinase 
 PGK: 3pg_c + atp_c <=> 13dpg_c + adp_c
PGM -11.92478591854981 Glycolysis/Gluconeogenesis Phosphoglycerate mutase 

In [92]:
rxn_list = []

for r in lst.reactions:
    sub = r.subsystem
    if sub =='Pentose Phosphate Pathway':
        rxn_list.append(r)

sorted_reactions = sorted(rxn_list, key=lambda reaction: reaction.id)

for r in sorted_reactions:
    if lst_glc_soln[r.id]!=0:
        print(r.id,lst_glc_soln[r.id],r.subsystem,r.name,'\n',r)

FBA3 2.2345260347653273 Pentose Phosphate Pathway Sedoheptulose 1,7-bisphosphate D-glyceraldehyde-3-phosphate-lyase 
 FBA3: s17bp_c <=> dhap_c + e4p_c
G6PDH2r 7.2025618459514815 Pentose Phosphate Pathway Glucose 6-phosphate dehydrogenase 
 G6PDH2r: g6p_c + nadp_c <=> 6pgl_c + h_c + nadph_c
GND 7.2025618459514815 Pentose Phosphate Pathway Phosphogluconate dehydrogenase 
 GND: 6pgc_c + nadp_c --> co2_c + nadph_c + ru5p__D_c
PFK_3 2.2345260347653273 Pentose Phosphate Pathway Phosphofructokinase (s7p) 
 PFK_3: atp_c + s7p_c --> adp_c + h_c + s17bp_c
PGL 7.2025618459514815 Pentose Phosphate Pathway 6-phosphogluconolactonase 
 PGL: 6pgl_c + h2o_c --> 6pgc_c + h_c
PPM -0.0032838105833473807 Pentose Phosphate Pathway Phosphopentomutase 
 PPM: r1p_c <=> r5p_c
RPE 4.275391735247468 Pentose Phosphate Pathway Ribulose 5-phosphate 3-epimerase 
 RPE: ru5p__D_c <=> xu5p__D_c
RPI -2.9250368591272937 Pentose Phosphate Pathway Ribose-5-phosphate isomerase 
 RPI: r5p_c <=> ru5p__D_c
TKT1 2.23452603476532

In [93]:
lst_glc_soln.fluxes['GND']

7.2025618459514815

In [94]:
lst_glc_soln.fluxes['PGK']

-13.298208262763724

In [95]:
lst_glc_soln.fluxes['CSm']

8.232561612229778

# Repeat for NADH.

In [96]:
lst_glc_soln = pfba(lst)

In [97]:

soln = pfba(lst)#optimize()


# Get the percentage contributions
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["nadh_c", "nadh_m", "nadh_x"],soln,lst)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADH production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():

    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total NADH consumption.")

Reaction GAPD contributes 28.57% of the total NADH production.
Reaction GCALDD contributes 0.01% of the total NADH production.
Reaction PGCD contributes 2.95% of the total NADH production.
Reaction FDH contributes 0.92% of the total NADH production.
Reaction IMPD contributes 0.11% of the total NADH production.
Reaction SACCD2 contributes 0.32% of the total NADH production.
Reaction HISTD contributes 0.24% of the total NADH production.
Reaction IPMD contributes 0.56% of the total NADH production.
Reaction OIVD2m contributes 0.27% of the total NADH production.
Reaction MMSAD1m contributes 0.27% of the total NADH production.
Reaction MMSAD3m contributes 0.27% of the total NADH production.
Reaction HICITDm contributes 0.32% of the total NADH production.
Reaction 3HPDm contributes 0.27% of the total NADH production.
Reaction ICDHxm contributes 6.66% of the total NADH production.
Reaction PDHm contributes 16.26% of the total NADH production.
Reaction AKGDm contributes 5.04% of the total NADH

# look at ATP. 

In [98]:
lst_glc_soln = pfba(lst)


soln = pfba(lst)#optimize()


# Get the percentage contributions
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["atp_c", "atp_m", "atp_x"],soln,lst)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total ATP production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():

    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total ATP consumption.")
# rxn_list = []
# for r in lst.metabolites.atp_c.reactions:
#     if lst_glc_soln[r.id]!=0:
#         rxn_list.append(r)
# #         print(r.id,lst_glc_soln[r.id],r.subsystem,r.name)

# for r in lst.metabolites.atp_m.reactions:
#     if lst_glc_soln[r.id]!=0:
#         rxn_list.append(r)
# #         print(r.id,lst_glc_soln[r

# for r in lst.metabolites.atp_x.reactions:
#     if lst_glc_soln[r.id]!=0:
#         rxn_list.append(r)
        
# sorted_reactions = sorted(rxn_list, key=lambda reaction: reaction.id)

# for r in sorted_reactions:
#     print(r.id,lst_glc_soln[r.id],r.subsystem,r.name)

Reaction PYK contributes 5.22% of the total ATP production.
Reaction ATPtm contributes 51.21% of the total ATP production.
Reaction PGK contributes 6.02% of the total ATP production.
Reaction URIDK2r contributes 0.00% of the total ATP production.
Reaction SUCOASm contributes 1.06% of the total ATP production.
Reaction ATPS3m contributes 50.65% of the total ATP production.


Reaction GLYTRS contributes 0.16% of the total ATP consumption.
Reaction RBFK contributes 0.00% of the total ATP consumption.
Reaction ATPM contributes 0.15% of the total ATP consumption.
Reaction MEVK1 contributes 0.03% of the total ATP consumption.
Reaction METTRS contributes 0.02% of the total ATP consumption.
Reaction PNTK contributes 0.00% of the total ATP consumption.
Reaction BIOMASS_Ls contributes 34.19% of the total ATP consumption.
Reaction PFK_3 contributes 0.89% of the total ATP consumption.
Reaction HEX1 contributes 3.96% of the total ATP consumption.
Reaction ASPK contributes 0.18% of the total ATP con

In [99]:
lst.reactions.ADK1

Reaction identifier,ADK1
Name,Adenylate kinase
Memory address,0x17ca2f1d0
Stoichiometry,amp_c + atp_c <=> 2.0 adp_c AMP C10H12N5O7P + ATP C10H12N5O13P3 <=> 2.0 ADP C10H12N5O10P2
GPR,Lipst1_1_2516 or Lipst1_1_327549
Lower bound,-1000.0
Upper bound,1000.0


In [100]:
sce.reactions.ADK1

Reaction identifier,ADK1
Name,adenylate kinase
Memory address,0x289f50dd0
Stoichiometry,amp_c + atp_c --> 2.0 adp_c AMP + ATP --> 2.0 ADP
GPR,YDL166C or YDR226W
Lower bound,0.0
Upper bound,1000.0


#### introduced constraint.

In [101]:
lst.reactions.ADK1.lower_bound = 0

In [102]:
lst_glc_soln = pfba(lst)


soln = pfba(lst)#optimize()
print(soln['FBA'])
print(soln['GND'])

# Get the percentage contributions
nadph_produce,nadph_consume = get_reaction_contribution_to_metabolite_production(["atp_c", "atp_m", "atp_x"],soln,lst)

for rxn_id, percentage in nadph_produce.items():
    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total ATP production.")

print('\n')
for rxn_id, percentage in nadph_consume.items():

    print(f"Reaction {rxn_id} contributes {percentage:.2f}% of the total ATP consumption.")


3.279276568507654
7.2025618459505045
Reaction PYK contributes 5.22% of the total ATP production.
Reaction ATPtm contributes 51.21% of the total ATP production.
Reaction PGK contributes 6.02% of the total ATP production.
Reaction URIDK2r contributes 0.00% of the total ATP production.
Reaction SUCOASm contributes 1.06% of the total ATP production.
Reaction ATPS3m contributes 50.65% of the total ATP production.


Reaction GLYTRS contributes 0.16% of the total ATP consumption.
Reaction RBFK contributes 0.00% of the total ATP consumption.
Reaction ATPM contributes 0.15% of the total ATP consumption.
Reaction MEVK1 contributes 0.03% of the total ATP consumption.
Reaction METTRS contributes 0.02% of the total ATP consumption.
Reaction PNTK contributes 0.00% of the total ATP consumption.
Reaction BIOMASS_Ls contributes 34.19% of the total ATP consumption.
Reaction PFK_3 contributes 0.89% of the total ATP consumption.
Reaction HEX1 contributes 3.96% of the total ATP consumption.
Reaction ASPK c

leave one reaction unconstrained for ICDH - leaves room for more realistic flux. 

In [106]:
lst_glc_soln = pfba(lst)

In [107]:
# lst_glc_soln.fluxes.to_csv('forFig_pub_nogludglusx.csv')

In [108]:
cobra.io.save_json_model(lst,'../models/Lst_v0.3_model_forPUB.json')
